In [42]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
# Конфигурация устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [44]:
# Определение класса модели
class TextClassifier(nn.Module):
    def __init__(self, n_classes):
        super(TextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased', return_dict=False)
        self.dropout = nn.Dropout(p=0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        output = self.dropout(outputs[1])  # берем pooled output
        return self.classifier(output)

In [ ]:
# Загрузка модели и компонентов
n_classes = 12
model = TextClassifier(n_classes)
model.load_state_dict(torch.load('model_CFO_sd.pth', map_location=device))
model.to(device)
model.eval()

In [46]:
with open('label_encoder.pkl', 'rb') as file:
    le = pickle.load(file)


In [47]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

# Загрузка данных и предварительная обработка
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Geo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).lower()
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

In [87]:
link = pd.read_excel('D:\\ProgPrj\\dsProjects\\ЦП_ЮФО_Подготовка\\datasetTMP.xlsx')
link = link.drop(columns="Название продукта")
link

,Описание,Технологии,Профессия,Ссылка на продукт
0,приглашаем удаленный формат работы программист...,1c soap http xml json скд,программист 1с 1с разработчик профессия стыке ...,https://gb.ru/geek_university/developer/progra...
1,предстоит заниматься работа scrum agile разраб...,1c soap http xml json скд,программист 1с 1с разработчик профессия стыке ...,https://gb.ru/geek_university/developer/progra...
2,ооо рт информ аккредитованная it компания инфр...,1c soap http xml json скд,программист 1с 1с разработчик профессия стыке ...,https://gb.ru/geek_university/developer/progra...
3,первый бит это команда программистов аналитико...,1c soap http xml json скд,программист 1с 1с разработчик профессия стыке ...,https://gb.ru/geek_university/developer/progra...
4,перспектива карьерного роста обязанности монит...,python linux tcp ip fhrp stp bgp ospf,инженер сетевым коммуникациям востребован комп...,https://gb.ru/geek_university/developer/archit...
5,компания zelax ищет молодых перспективных спец...,python linux tcp ip fhrp stp bgp ospf,инженер сетевым коммуникациям востребован комп...,https://gb.ru/geek_university/developer/archit...
6,wildberries крупнейший маркетплейс россии wild...,python linux tcp ip fhrp stp bgp ospf,инженер сетевым коммуникациям востребован комп...,https://gb.ru/geek_university/developer/archit...
7,присоединяйтесь команде сибур диджитал создаем...,python linux tcp ip fhrp stp bgp ospf,инженер сетевым коммуникациям востребован комп...,https://gb.ru/geek_university/developer/archit...
8,обязанности разработка внедрение поддержка тер...,python linux tcp ip fhrp stp bgp ospf,инженер сетевым коммуникациям востребован комп...,https://gb.ru/geek_university/developer/archit...
9,lamoda group это крупнейшая россии снг онлайн ...,python pycharm gitlab pytest sql mysql postgre...,python пишут сайты приложения игры чат боты на...,https://gb.ru/geek_university/developer/progra...


In [90]:
dataset = link

In [91]:
dataset

,Описание,Технологии,Профессия,Ссылка на продукт
0,приглашаем удаленный формат работы программист...,1c soap http xml json скд,программист 1с 1с разработчик профессия стыке ...,https://gb.ru/geek_university/developer/progra...
1,предстоит заниматься работа scrum agile разраб...,1c soap http xml json скд,программист 1с 1с разработчик профессия стыке ...,https://gb.ru/geek_university/developer/progra...
2,ооо рт информ аккредитованная it компания инфр...,1c soap http xml json скд,программист 1с 1с разработчик профессия стыке ...,https://gb.ru/geek_university/developer/progra...
3,первый бит это команда программистов аналитико...,1c soap http xml json скд,программист 1с 1с разработчик профессия стыке ...,https://gb.ru/geek_university/developer/progra...
4,перспектива карьерного роста обязанности монит...,python linux tcp ip fhrp stp bgp ospf,инженер сетевым коммуникациям востребован комп...,https://gb.ru/geek_university/developer/archit...
5,компания zelax ищет молодых перспективных спец...,python linux tcp ip fhrp stp bgp ospf,инженер сетевым коммуникациям востребован комп...,https://gb.ru/geek_university/developer/archit...
6,wildberries крупнейший маркетплейс россии wild...,python linux tcp ip fhrp stp bgp ospf,инженер сетевым коммуникациям востребован комп...,https://gb.ru/geek_university/developer/archit...
7,присоединяйтесь команде сибур диджитал создаем...,python linux tcp ip fhrp stp bgp ospf,инженер сетевым коммуникациям востребован комп...,https://gb.ru/geek_university/developer/archit...
8,обязанности разработка внедрение поддержка тер...,python linux tcp ip fhrp stp bgp ospf,инженер сетевым коммуникациям востребован комп...,https://gb.ru/geek_university/developer/archit...
9,lamoda group это крупнейшая россии снг онлайн ...,python pycharm gitlab pytest sql mysql postgre...,python пишут сайты приложения игры чат боты на...,https://gb.ru/geek_university/developer/progra...


In [92]:

def predict_url(description):
    encoded = tokenizer.encode_plus(
        description, add_special_tokens=True, max_length=512,
        padding='max_length', truncation=True, return_attention_mask=True,
        return_tensors='pt')
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        _, predicted_index = torch.max(outputs, dim=1)
    return le.inverse_transform([predicted_index.item()])[0]

In [93]:

def recommend_product(user_description, dataset):
    tfidf_desc = TfidfVectorizer()
    tfidf_matrix_desc = tfidf_desc.fit_transform(dataset["Описание"])
    cleaned_query = clean_text(user_description)
    query_vector_desc = tfidf_desc.transform([cleaned_query])
    cos_sim_desc = cosine_similarity(query_vector_desc, tfidf_matrix_desc)
    enhanced_cos_sim = cos_sim_desc * 2  # Увеличиваем влияние TF-IDF
    best_idx = enhanced_cos_sim.argmax()
    return dataset.iloc[best_idx]['Ссылка на продукт']



In [106]:

# Пользовательский ввод и вывод предсказаний
user_description = ""

In [107]:
predicted_url = predict_url(user_description)
recommended_product = recommend_product(user_description, dataset)

In [108]:
# Выбор наиболее предпочтительного ответа
ensemble_output = predicted_url if predicted_url == recommended_product else recommended_product

In [110]:

print(f"Вывод ансамбля: {ensemble_output}")

Вывод ансамбля: https://gb.ru/geek_university/developer/programmer/ios
